In [ ]:

!pip install pulp
from pulp import *
import pandas as pd

In [ ]:
dream_teams = pd.DataFrame(columns=['name', 'position', 'GW','predictions',"team","opponent_team","value"])
#Read file of predictions
optimize_df = pd.read_csv("new_optimize_df_to_gw9 (1).csv")
optimize_df = optimize_df.dropna()
optimize_df = optimize_df.drop_duplicates(subset=["name","GW"],keep='first').reset_index()


optimize_df


,index,Unnamed: 0,predictions,name,GW,team,value,position,opponent_team
0,0,0,4.06,Alex McCarthy,4,Southampton,45,GK,18
1,1,1,3.65,Emiliano Martínez,4,Aston Villa,46,GK,11
2,2,2,3.45,Kasper Schmeichel,4,Leicester,55,GK,19
3,3,3,3.33,Illan Meslier,4,Leeds,45,GK,12
4,4,4,3.14,Rui Pedro dos Santos Patrício,4,Wolves,55,GK,8
...,...,...,...,...,...,...,...,...,...
2951,3220,3220,1.55,Michail Antonio,9,West Ham,62,FWD,15
2952,3221,3221,1.09,Joelinton Cássio Apolinário de Lira,9,Newcastle,57,FWD,5
2953,3222,3222,1.05,Sergio Agüero,9,Man City,103,FWD,17
2954,3223,3223,0.99,Rodrigo Moreno,9,Leeds,58,FWD,1


In [ ]:
#Remove 100% sure non-playing players 
pdata9 = optimize_df[optimize_df["GW"]==9]
print(pdata9.shape)
pdata9 = pdata9[pdata9["name"] != "Mohamed Salah"]
print(pdata9.shape)



pdata7 = optimize_df[optimize_df["GW"]==7]
print(pdata7.shape)
pdata7 = pdata7[pdata7["name"] != "Timothy Castagne"]
print(pdata7.shape)


pdata5 = optimize_df[optimize_df["GW"]==5]
print(pdata5.shape)
pdata5 = pdata5[pdata5["name"] != "Kevin De Bruyne"]
pdata5 = pdata5[pdata5["name"] != "Çaglar Söyüncü"]

print(pdata5.shape)




(293, 9)
(292, 9)
(562, 9)
(561, 9)
(539, 9)
(537, 9)


In [ ]:


#Run pip install cell for each GW, or the model crashes.
pdata = optimize_df[optimize_df["GW"]==4]
print(pdata)
pdata = pdata.reset_index()
prob = pulp.LpProblem('FantasyTeam2',LpMaximize)
#Decision variables(x0,x1...xN)
decision_variables = []
for rownum, row in pdata.iterrows():
    variable = str('x' + str(rownum))
    variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer') #make variables binary
    decision_variables.append(variable)

print ("Total number of decision_variables: " + str(len(decision_variables)))
# Returns: Total number of decision_variables
#Optimization function
total_points = ""
for rownum, row in pdata.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            formula = row['predictions']*player
            total_points += formula
            
prob += total_points
print ("Optimization function: " + str(total_points))
  #Cash constraints
avail_cash = 830
total_paid = ""
for rownum, row in pdata.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            formula = row['value']*player
            total_paid += formula

prob += (total_paid <= avail_cash)
  #GK constraint
avail_gk = 1
total_gk = ""
for rownum, row in pdata.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            if row['position'] == 'GK':
                formula = 1*player
                total_gk += formula
prob += (total_gk == avail_gk)
#print(total_gk)
avail_def = 3
total_def = ""
for rownum, row in pdata.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            if row['position'] == 'DEF':
                formula = 1*player
                total_def += formula
prob += (total_def == avail_def)
#print((total_def))
avail_mid = 4
total_mid = ""
for rownum, row in pdata.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            if row['position'] == 'MID':
                formula = 1*player
                total_mid += formula
prob += (total_mid == avail_mid)
#print((total_mid))
avail_fwd = 3
total_fwd = ""
for rownum, row in pdata.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            if row['position'] == 'FWD':
                formula = 1*player
                total_fwd += formula
prob += (total_fwd == avail_fwd)
#print(total_fwd)
team_dict= {}
for team in set(pdata.team):
    team_dict[str(team)]=dict()
    team_dict[str(team)]['avail'] = 3
    team_dict[str(team)]['total'] = ""
    for rownum, row in pdata.iterrows():
        for i, player in enumerate(decision_variables):
            if rownum == i:
                if row['team'] == team:
                    formula = 1*player
                    team_dict[str(team)]['total'] += formula

    prob += (team_dict[str(team)]['total'] <= team_dict[str(team)]['avail'])
#print(len(team_dict))
prob.writeLP('FantasyTeam.lp')
optimization_result = prob.solve()
#print("Status:", LpStatus[prob.status])
print("Optimal Solution to the problem: ", value(prob.objective))
variable_name = []
variable_value = []

for v in prob.variables():
    variable_name.append(v.name)
    variable_value.append(v.varValue)

df = pd.DataFrame({'variable': variable_name, 'value': variable_value})
for rownum, row in df.iterrows():
    value = re.findall(r'(\d+)', row['variable'])
    df.loc[rownum, 'variable'] = int(value[0])

df = df.sort_values('variable')

#append results
for rownum, row in pdata.iterrows():
    for results_rownum, results_row in df.iterrows():
        if rownum == results_row['variable']:
            pdata.loc[rownum, 'decision'] = results_row['value']


pdata[pdata.decision==1].predictions.sum() # Returns 2010.8606251232461

pdata[pdata.decision==1].value.sum() # Returns 830
pdata[pdata.decision==1].predictions.sum()

print("Team cost: ",pdata[pdata.decision==1].value.sum())
dream_teams = pd.concat([dream_teams,pdata[pdata.decision==1][["name","GW","predictions","team","opponent_team","position","value"]]])



     index  Unnamed: 0  predictions  ... value  position opponent_team
0        0           0         4.06  ...    45        GK            18
1        1           1         3.65  ...    46        GK            11
2        2           2         3.45  ...    55        GK            19
3        3           3         3.33  ...    45        GK            12
4        4           4         3.14  ...    55        GK             8
..     ...         ...          ...  ...   ...       ...           ...
420    543         543         0.85  ...    55       FWD             9
421    544         544         0.85  ...    55       FWD             9
422    545         545         0.85  ...    49       FWD            16
423    546         546         0.85  ...    49       FWD             7
424    547         547         0.85  ...    49       FWD             7

[425 rows x 9 columns]
Total number of decision_variables: 425
Optimization function: 4.06*x0 + 3.65*x1 + 2.87*x10 + 1.88*x100 + 1.87*x101 + 1.87*x

In [ ]:
#Show dream teams

dream_teams

,name,position,GW,predictions,team,opponent_team,value
0,Alex McCarthy,GK,4,4.06,Southampton,18,45
47,Ryan Bertrand,DEF,4,5.04,Southampton,18,50
50,Kyle Walker-Peters,DEF,4,4.83,Southampton,18,45
51,Reece James,DEF,4,4.76,Chelsea,6,51
52,Ben Davies,DEF,4,4.74,Spurs,13,49
53,James Justin,DEF,4,4.67,Leicester,19,47
195,Heung-Min Son,MID,4,8.95,Spurs,13,90
197,Harvey Barnes,MID,4,7.19,Leicester,19,71
200,Wilfried Zaha,MID,4,6.44,Crystal Palace,5,72
203,Dennis Praet,MID,4,5.12,Leicester,19,55


In [ ]:
#Save teams for use in website
dream_teams.to_csv('/content/dream_team_df_gw4-9_new.csv')
